# Homework 4

## FINM 37500 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

You will use a single row of this data.
* date: `2023-05-01`
* expiration: 1yr
* tenor: 2yrs

## Rate Data

The file `data/cap_quotes_2023-05-01.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has annual payment frequency
* as mentioned above, expiration is 1 year and tenor of the underlying swap is 2 years. Thus, a so-called 1x2 swaption where the underlying swap begins in 1 year from now and ends 3 years from now.
* strike is ATM, namely, at the forward swap rate
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

# 1. Black's Formula for Swaptions

Use Black's formula to price the swaption.

This is done much as we used Black's formula for caps and caplets. Except...

1. No need to worry about timing issues. 
    * The swaption expires at $T_o=1$ and that is also when the uncertainty is resolved. So use this value in the formula. 
    * Recall that for caplets, we had to use a period before the payment due to the deferred timing of the caplet's underlying and payoff.

2. You only need to use Black's formula once.
    * With caps, we had to use Black's formula repeatedly to price each caplet.
    * A swaption is a single option. However, it involves (if exercised) multiple payments at different times. 
    

### Discount factor
There is one important difference for pricing swaptions using Black's formula:
* For other derivatives, (such as caplets,) we used the discount factor, $Z(0,T_o)$ in Black's formula to discount the option payoff.
* For a swaption, we instead use the sum of discount factors corresponding to each swap payment:
$$Z^{\text{swap}}(0,T_o,T) \equiv \sum_{i=1}^mZ(0,T_i)$$
where $T_i$ for $1\le i \le m$ denotes the swap payment dates.

Thus, for the swaption with $T_o=1, T_s=2, T=3$ written on an **annually paying swap**, we have
$$Z^{\text{swap}}(0,T_o,T) = Z(0,2) + Z(0,3)$$

### Forward Swap Rate
In Black's formula we need the **forward** on the underlying, not the underlying rate itself.
* We need the **forward swap rate**, not the (spot) swap rate.

The forward swap rate is calculated from spot swap rates (provided in the data) similarly to how we calculate forward interest rates from spot rates.

$$\newcommand{\Tfwd}{T_{\text{fwd}}}$$
$$\newcommand{\fwdswap}{s(t,\Tfwd,T)}$$
$$\fwdswap \equiv n\frac{Z(t,\Tfwd)-Z(t,T)}{\sum_{i=1}^mZ(t,T_i)}$$
where 
* $Z$ denotes the usual discount factor.
* $n$ is the frequency of the swap payments. (That is, $n=2$ for semiannual swaps, etc.)
* $T_i$ for $1\le i\le m$ again denotes the time to each swap payment, with $T_m=T$.

## 1.1
Calculate the forward swap rate for $\Tfwd=1$ and $T=3$, noting that the 1x2 swaption has an underlying swap terminating 3 years from today, not 2 years from today.

## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 2yr swaption
* The 1yr x 4yr swaption

Report these values and compare to the price of the 1yr x 2yr swaption priced in the previous problem.

(In the rest of the homework, we only focus on the 1yr x 2yr swaption. These alternate swaptions are not used further.)

# 2. SABR Volatility Modeling

Continue considering the 1y2y swaption described in the previous section.

Use the quoted volatility skew to fit a SABR model.

Throughout, parameterize, `$\beta=.75$`.

## 2.1
Estimate $(\alpha,\rho,\nu)$ via the SABR formula in W.5. Feel free to use the `sabr` function in `cmds/volskew.py`.

Report the values of these parameters.

## 2.2
Create a grid of strikes of [.0025, .09], with grid spacing of 10bps, (.0010).

Use the SABR model parameterized above to calculate the volatility for each of these strikes.

Plot the SABR curve, and also include the market quotes in the plot.

Conceptually, does the SABR curve fit these points well? Perfectly?

## 2.3
Suppose we want to price a 1y2y swaption but with a far out-of-the-money strike of 6\%. 

Note that this strike is far outside the range for which we have market quotes, and even if we do have quotes, they likely are not liquid on a given day.

Use the SABR vol at this 6\% strike to price the swaption.


## 2.4
*Optional*

Use the ATM implied volatility to fit $\alpha$, (sometimes denoted $\sigma_0$.) That is, for any choice of $(\rho,\nu)$, solve a quadratic equation to get $\alpha$ as a function of those two parameters, along with $\beta$, which is at its ad-hoc value.

Recall that we have a simple relationship between ATM vol and $\alpha$.
$$\sigma_\text{ATM} = \alpha\frac{B}{F^{1-\beta}}$$
where $B$ is defined in W.5. It is a quadratic expression of $\alpha$ along with $(\beta,\rho,\nu)$.

This decouples the optimization. We solve for $(\rho,\nu)$ and $\alpha$ as a function of the other two parameters.

Consider using the function `sabrATM` in `cmds/volskew.py`.

# 3. SABR and Risk Management

We are considering May 1, 2023. The next two days the Board of Governors of the Federal Reserve met.

We consider how an **increase** of 25bps in the underlying rate impacts the price of the 1y2y ATM swaption.

Here, we are assuming that the Fed's decision
* will impact the forward swap rate directly, one-for-one with the Fed Funds rate.
* the shift happens on May 3, 2023.

Of course, in reality, the Fed's decision
* may price in early to some degree or not settle at the Fed's target
* may not impact the forward swap rate the same as the Fed Funds rate. We would need to model how the discount curve changes and what that change (level, slope, curvature?) would do to the forward swap rate.

## 3.1

Use Black's equation to re-price the 1y2y swaption with
* the same ATM volatility
* an underlying (forward swap) rate increased 25 bps.
* the same time-to-maturities. (We should decrease these all by 2/365, but we focus here on the delta effects rather than the theta. And it would be a small impact anyone.)

Report 
* the new price
* the change in price divided by 25bps, (a numerical delta.)

## 3.2

Now, we consider how the volatility may change with the underlying shift of 25bps.

Using the same SABR parameters from 2.1, and the strike grid from 2.2, plot the new SABR curve accounting for the underlying rate shifting up 25bps.

## 3.3

Calculate the new volatility specifically for the pre-shift ATM strike.

Use this in Black's formula similar to 3.1, where the only change is the volatility now reflects the shift in the underlying (not the strike).

Report
* the new price
* the change in price divided by 25bps

## 3.4

How much different is the (dynamic) delta which accounts for the shift in volatility from the (static) delta in 3.1?

***